In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from NegativeClassOptimization import ml
from NegativeClassOptimization import utils
from NegativeClassOptimization import preprocessing

/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
df = pd.read_csv(
    "../data/mutants/mutant_igs/1ADQ__vs__1FBI_mut_igs.csv"
)
print(df.shape)
df.head()

(1882, 18)


,ID_slide_Variant,CDR3,Best,Slide,Energy,Structure,n_mut,ID,original Slide,IG org,Energy org,mut OH,IG mut,degree wt,posn IG mut,posn IG org,posn delta IG,dEnergy
0,SHH17266801_02a_1m_2_01a,CAALLLRNWYFDVW,False,AALLLRNWYFD,-102.20,124837-UULLSRDSDL,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0...","[0.06076978215228966, 0.01406735472193295, -0....",2,0.060770,0.055034,-0.005736,1.16
1,SHH17266801_02a_1m_3_01a,CCALLLRNWYFDVW,False,CALLLRNWYFD,-104.04,124837-UULLSRDSDL,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0...","[0.015935059620197915, 0.027848667955512114, -...",2,0.015935,0.055034,0.039099,3.00
2,SHH17266801_02a_1m_4_01a,CDALLLRNWYFDVW,False,DALLLRNWYFD,-84.95,124772-ULSDSLDLDD,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0...","[0.05790601755143533, 0.015108458679792963, -0...",2,0.057906,0.055034,-0.002872,-16.09
3,SHH17266801_02a_1m_5_01a,CEALLLRNWYFDVW,False,EALLLRNWYFD,-84.97,124772-ULSDSLDLDD,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0...","[0.04100764037843625, 0.020631313413037082, -0...",2,0.041008,0.055034,0.014026,-16.07
4,SHH17266801_02a_1m_6_01a,CFALLLRNWYFDVW,False,FALLLRNWYFD,-87.00,132961-DLDLRDRDDL,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0...","[-0.5975218928392115, 0.39205529997472094, -0....",2,-0.597522,0.055034,0.652556,-14.04


In [5]:
loader = datasets.FrozenMiniAbsolutMLLoader(data_dir=Path("../data/Frozen_MiniAbsolut_ML/"))
loader.load(task)
task

HIGH_VS_95LOW__5E94__auto__0__42

2. Convert for energy contribution calculation.

In [6]:
df.head()

,ID_slide_Variant,CDR3,Best,Slide,Energy,Structure,n_mut,ID,original Slide,IG org,Energy org,mut OH,IG mut,degree wt,posn IG mut,posn IG org,posn delta IG,dEnergy
0,SHH17266801_02a_1m_2_01a,CAALLLRNWYFDVW,False,AALLLRNWYFD,-102.20,124837-UULLSRDSDL,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0...","[0.06076978215228966, 0.01406735472193295, -0....",2,0.060770,0.055034,-0.005736,1.16
1,SHH17266801_02a_1m_3_01a,CCALLLRNWYFDVW,False,CALLLRNWYFD,-104.04,124837-UULLSRDSDL,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0...","[0.015935059620197915, 0.027848667955512114, -...",2,0.015935,0.055034,0.039099,3.00
2,SHH17266801_02a_1m_4_01a,CDALLLRNWYFDVW,False,DALLLRNWYFD,-84.95,124772-ULSDSLDLDD,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0...","[0.05790601755143533, 0.015108458679792963, -0...",2,0.057906,0.055034,-0.002872,-16.09
3,SHH17266801_02a_1m_5_01a,CEALLLRNWYFDVW,False,EALLLRNWYFD,-84.97,124772-ULSDSLDLDD,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0...","[0.04100764037843625, 0.020631313413037082, -0...",2,0.041008,0.055034,0.014026,-16.07
4,SHH17266801_02a_1m_6_01a,CFALLLRNWYFDVW,False,FALLLRNWYFD,-87.00,132961-DLDLRDRDDL,1.0,17266801_02a,TALLLRNWYFD,"[0.05503381665020039, 0.015365579785776812, -0...",-101.04,"[[array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0...","[-0.5975218928392115, 0.39205529997472094, -0....",2,-0.597522,0.055034,0.652556,-14.04


In [ ]:
for path in Path("../data/mutants/mutant_igs/").glob("*.csv"):
    if "shuffled" in str(path):
        continue
    str_describing_task = path.name.split("_mut_igs")[0]

    if "__vs__" in str_describing_task:
        # ONE_VS_ONE: compute once for positive and once for negative antigen.
        raise NotImplementedError()
    else:
        antigen = str_describing_task[:4]

    antigen_name_row = f"#Antigen {antigen}"

    break

In [11]:
df_ = df.iloc[:, :6]
df_.columns = [
    "ID",
    "CDR3Sequence",
    "BestForThisCDR?('true'/'false')",
    "Slide",
    "Energy",
    "Structure",
]
df_["BestForThisCDR?('true'/'false')"] = df_[
    "BestForThisCDR?('true'/'false')"
].replace({True: "true", False: "false"})
df_.head()

,ID,CDR3Sequence,BestForThisCDR?('true'/'false'),Slide,Energy,Structure
0,SHH17266801_02a_1m_2_01a,CAALLLRNWYFDVW,false,AALLLRNWYFD,-102.20,124837-UULLSRDSDL
1,SHH17266801_02a_1m_3_01a,CCALLLRNWYFDVW,false,CALLLRNWYFD,-104.04,124837-UULLSRDSDL
2,SHH17266801_02a_1m_4_01a,CDALLLRNWYFDVW,false,DALLLRNWYFD,-84.95,124772-ULSDSLDLDD
3,SHH17266801_02a_1m_5_01a,CEALLLRNWYFDVW,false,EALLLRNWYFD,-84.97,124772-ULSDSLDLDD
4,SHH17266801_02a_1m_6_01a,CFALLLRNWYFDVW,false,FALLLRNWYFD,-87.00,132961-DLDLRDRDDL
